In [182]:
import requests
import json
import pandas as pd
import re
import emoji
import flair

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', None)


ApiKey = 'zq851ydtQ1XQi5UdNSMUPPInz'
ApiKeySecret = 'pKPlz171RVpaLlEESxDmRTvxuk452M8GEV92c786twfcuTs9y4'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAD3iVwEAAAAAp8IJMUXWf1wxFWiZfhACIHQXqSI%3DEOjIMXIXNZnXwXMpOi7p3ecqlMW7iOhwS8Dmbk5htotAzSFQmG'


def response():
    endpoint = 'https://api.twitter.com/2/tweets/search/recent'
    headers = {'authorization': f'Bearer {bearer_token}'}
    params = {
        'query': '(tesla) (lang:en)', #add (tesla OR tsla)
        'max_results': '100',
        'tweet.fields': 'created_at,lang',
        'author_id' : '807095'
    }
    response = requests.get(endpoint,
                                params=params,
                                headers=headers)  # send the request
    return response

In [183]:
response = response()
data = response.json()['data']
df = pd.DataFrame.from_dict(data)

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

  
def cleanTweet(tweet):
    if 'RT' in tweet[:3]:
        tweet = tweet.replace('RT','')
    tweet = remove_emojis(tweet)
    tweet = tweet.replace(':','')
    tweet = re.sub("@[A-Za-z0-9]+","",tweet) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI) #Remove Emojis
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    return tweet

def cleanDf(columnName):
    cleanList = []
    for tweet in df[columnName]:
        clean_tweet = cleanTweet(tweet)
        cleanList.append(clean_tweet)
    df[columnName] = cleanList

cleanDf('text')

sentiment_model = flair.models.TextClassifier.load('en-sentiment')


KeyError: 'data'

In [180]:
# text = df['text'].iloc[0]
# sentence = flair.data.Sentence(text)
# sentiment_model.predict(sentence)
# sentence


def addSentiment ():
    sentimentColumnN = []
    sentimentColumnL = []
    for tweet in df['text']:
        sentence = flair.data.Sentence(tweet)
        sentiment_model.predict(sentence)
        split_sentence = str(sentence.labels[0]).split(' ')
        numeric_score = split_sentence[1]
        label_score = split_sentence[0]
        sentimentColumnN.append(numeric_score)
        sentimentColumnL.append(label_score)
    df['Sentiment Outcome'] = sentimentColumnL
    df['Confidence Score'] = sentimentColumnN


addSentiment()

In [181]:
df

,lang,id,text,created_at,Sentiment Outcome,Confidence Score
0,en,1461783614308462601,Anti inflation ShibaFloki Nextshiba NewShiba shiba have nft swap burned 77% tesla in phase 5 reward 5 % burning 5% 1/4 Shibainu have token gate soon bitcoin eth floki bnb binance ELON shibaarmy hero nextshiba newshiba crypto ETH,2021-11-19T19:49:06.000Z,NEGATIVE,(0.9997)
1,en,1461783603206135812,Tesla subjects women to 'nightmarish' sexual harassment at factory - lawsuit - Reuters https//t.co/VIwyc9G7jD,2021-11-19T19:49:04.000Z,NEGATIVE,(0.9911)
2,en,1461783602870505476,9⃣2⃣KHolderssss Hold and earn BTC Times Square Advertisements elon mus…,2021-11-19T19:49:04.000Z,NEGATIVE,(0.8751)
3,en,1461783600324567043,io Anti inflation ShibaFloki Nextshiba NewShiba shiba have nft swap burned 77% tesla in phase 5 reward 5 % burning 5% 1/4 Shibainu have token gate soon bitcoin eth floki bnb binance ELON shibaarmy hero nextshiba newshiba crypto ETH https//t.co/lwveDWXK0G,2021-11-19T19:49:03.000Z,NEGATIVE,(0.998)
4,en,1461783592682541060,Saw a Model S Plaid in inventory Reserved using Apple Pay Paid online Drove 2.5 hours to pick it up I’m now an inc…,2021-11-19T19:49:01.000Z,NEGATIVE,(0.9997)
5,en,1461783589905960962,"The biggest community driven BSC project is giving away a brand new Tesla Model 3!As Elon said ""Hamster has mad skillz!!"". Join the madness now https//t.co/zh5d4qvPxk Hamstercoin HamsterPower TeslaGiveaway",2021-11-19T19:49:01.000Z,POSITIVE,(0.637)
6,en,1461783577008517125,io Anti inflation ShibaFloki Nextshiba NewShiba shiba have nft swap burned 77% tesla in phase 5 reward 5 % burning 5% 1/4 Shibainu have token gate soon bitcoin eth floki bnb binance ELON shibaarmy hero nextshiba newshiba crypto ETH https//t.co/EqotT9btvK,2021-11-19T19:48:58.000Z,NEGATIVE,(0.9988)
7,en,1461783573745311746,Hamster Reward Tesla BTCB Hamstercoin $HAM Top 10 list No 1. https//t.co/aY88y30DAJ,2021-11-19T19:48:57.000Z,NEGATIVE,(0.9963)
8,en,1461783568389230593,Still many people are unaware that Tesla started because many big car companies were shutting down their EV programs. Tesla d…,2021-11-19T19:48:55.000Z,NEGATIVE,(0.5153)
9,en,1461783565289590800,Anti inflation ShibaFloki Nextshiba NewShiba shiba have nft swap burned 77% tesla in phase 5 reward 5 % burning 5% 1/4 Shibainu have token gate soon bitcoin eth floki bnb binance ELON shibaarmy hero nextshiba newshiba crypto ETH,2021-11-19T19:48:55.000Z,NEGATIVE,(0.9997)


In [184]:
endpoint = 'https://api.twitter.com/2/tweets/search/recent'
headers = {'authorization': f'Bearer {bearer_token}'}
params = {
    'query': '(tesla) (lang:en)', #add (tesla OR tsla)
    'max_results': '10',
    'tweet.fields': 'created_at,lang',
    'author_id' : '807095'
}
response = requests.get(endpoint,
                            params=params,
                            headers=headers)  # send the request
response.json()

{'errors': [{'parameters': {'author_id': ['807095']},
   'message': 'The query parameter [author_id] is not one of [query,start_time,end_time,since_id,until_id,max_results,next_token,expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields]'}],
 'title': 'Invalid Request',
 'detail': 'One or more parameters to your request was invalid.',
 'type': 'https://api.twitter.com/2/problems/invalid-request'}